In [23]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn import tree
import graphviz

In [2]:
col_to_use =['location'
             ,'date'
             ,'total_cases_per_million'
#              ,'total_cases'
#              ,'new_cases'
             ,'new_cases_per_million'
             ,'new_cases_smoothed'
             ,'new_cases_smoothed_per_million'
             ,'new_deaths'
             ,'new_deaths_smoothed'
             ,'new_deaths_smoothed_per_million'
             ,'weekly_hosp_admissions_per_million'
             ,'weekly_icu_admissions_per_million'
             ,'new_vaccinations_smoothed_per_million'
             ,'people_vaccinated_per_hundred'
             ,'people_fully_vaccinated_per_hundred'
             ,'total_vaccinations' 
            ,'people_vaccinated'                        
            ,'people_fully_vaccinated'                 
            ,'new_vaccinations'                     
            ,'new_vaccinations_smoothed'
            ,'stringency_index'
#             ,'population'
            ,'population_density'
             ,'handwashing_facilities'
             ,'cardiovasc_death_rate'
             ,'diabetes_prevalence'
            ,'median_age'
            ,'aged_65_older'
            ,'aged_70_older'
            ,'gdp_per_capita'
            ,'extreme_poverty'
            ,'female_smokers'
            ,'male_smokers'
            ,'hospital_beds_per_thousand'
            ,'life_expectancy'
             ,'human_development_index'
            ]

In [3]:
df = pd.read_csv("owid-covid-data.csv",usecols =col_to_use)

In [4]:
# df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72673 entries, 0 to 72672
Data columns (total 34 columns):
location                                 72673 non-null object
date                                     72673 non-null object
new_cases_smoothed                       70411 non-null float64
new_deaths                               62233 non-null float64
new_deaths_smoothed                      70411 non-null float64
total_cases_per_million                  71366 non-null float64
new_cases_per_million                    71359 non-null float64
new_cases_smoothed_per_million           70411 non-null float64
new_deaths_smoothed_per_million          70411 non-null float64
weekly_icu_admissions_per_million        697 non-null float64
weekly_hosp_admissions_per_million       1224 non-null float64
total_vaccinations                       4411 non-null float64
people_vaccinated                        3898 non-null float64
people_fully_vaccinated                  2557 non-null float64
new

In [5]:
df[['new_cases_smoothed','new_vaccinations_smoothed']].corr()

,new_cases_smoothed,new_vaccinations_smoothed
new_cases_smoothed,1.000000,0.461335
new_vaccinations_smoothed,0.461335,1.000000


### Do Vaccine Brands affect COVID cases and deaths?
To explore if different vaccine brands have different effects on the containment and spread of the virus

In [6]:
vaccinebrand =  pd.read_csv("VaccineBrands.csv")

In [7]:
vaccinebrand.head()

,Location,Source,Last observation date,Vaccines
0,Albania,Ministry of Health,"Mar. 10, 2021",Pfizer/BioNTech
1,Algeria,Ministry of Health,"Feb. 19, 2021",Sputnik V
2,Andorra,Government of Andorra,"Mar. 10, 2021",Pfizer/BioNTech
3,Angola,Ministry of Health,"Mar. 8, 2021",Oxford/AstraZeneca
4,Anguilla,Ministry of Health,"Feb. 26, 2021",Oxford/AstraZeneca


In [8]:
#explode converts concatenated vaccine brands into separate rows
vaccinebrand = vaccinebrand.assign(Vaccines=vaccinebrand['Vaccines'].str.split(',')).explode('Vaccines')
vaccinebrand['count'] =1
#standardize formatting before pivoting to ensure each vaccine brand takes up single column
vaccinebrand['Vaccines'] = vaccinebrand['Vaccines'].apply(lambda x: x.strip().replace(" ",""))

In [9]:
#each row becomes a column/feature
vaccinebrand = vaccinebrand.pivot_table('count', ['Location'], 'Vaccines').reset_index()

In [10]:
vaccinebrand.fillna(0, inplace =True)
vaccinebrand.head()

Vaccines,Location,Covaxin,EpiVacCorona,Johnson&Johnson,Moderna,Oxford/AstraZeneca,Pfizer/BioNTech,Sinopharm/Beijing,Sinopharm/Wuhan,Sinovac,SputnikV
0,Albania,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Andorra,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Angola,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Anguilla,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# vaccinebrand.columns

In [12]:
#join the vaccine dataset to the main using location
dataframe = pd.merge(df, vaccinebrand, left_on='location', right_on='Location', how='left')
dataframe.drop(columns =['Location'], inplace=True)
dataframe.head()

,location,date,new_cases_smoothed,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,new_deaths_smoothed_per_million,weekly_icu_admissions_per_million,...,Covaxin,EpiVacCorona,Johnson&Johnson,Moderna,Oxford/AstraZeneca,Pfizer/BioNTech,Sinopharm/Beijing,Sinopharm/Wuhan,Sinovac,SputnikV
0,Afghanistan,2020-02-24,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2020-02-25,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2020-02-26,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2020-02-27,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2020-02-28,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Modelling new COVID cases using decision tree and OWID dataset + vaccine brands
- Removed vaccine brands due tolow feature importance score
- Removed unimportant variables / variables which appear to have spurious relationship

In [13]:
data = dataframe.fillna(0)
X = data[['new_vaccinations_smoothed_per_million'
          ,'people_vaccinated_per_hundred'
          ,'people_fully_vaccinated_per_hundred'
            ,'stringency_index'
            ,'population_density'
            ,'median_age'
            ,'aged_65_older'
            ,'aged_70_older'
            ,'gdp_per_capita'
            ,'extreme_poverty'
#             ,'female_smokers'
#             ,'male_smokers'
              ,'handwashing_facilities'
             ,'cardiovasc_death_rate'
             ,'diabetes_prevalence'
            ,'hospital_beds_per_thousand'
#             ,'life_expectancy'
             ,'human_development_index'
#          ,'Covaxin', 'EpiVacCorona', 'Johnson&Johnson', 'Moderna',
#        'Oxford/AstraZeneca', 'Pfizer/BioNTech',
#        'Sinopharm/Beijing', 'Sinopharm/Wuhan', 'Sinovac', 'SputnikV'
         ]]
y = data['new_cases_smoothed_per_million']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
for depth in range(5,35,5):
    print(depth)
    dt_tree = DecisionTreeRegressor(max_depth =depth,min_samples_split =20)
    dt_reg = dt_tree.fit(X_train, y_train)
    print(dt_reg.score(X_test, y_test))
    y_pred = dt_reg.predict(X_test)
    print(mean_squared_error(y_test, y_pred))

5
0.33696591871623816
13704.900782947672
10
0.5704626119731093
8878.528949940423
15
0.6919539200663276
6367.306117798414
20
0.7593794843297258
4973.6210953407835
25
0.7818888295209258
4508.353394565548
30
0.78135840041162
4519.317353313111


#### this code is used to generate a visualisation of the final decision tree moodel

In [24]:
dt_tree = DecisionTreeRegressor(max_depth =25, min_samples_leaf =50)
dt_reg = dt_tree.fit(X_train, y_train)
    
dot_data = tree.export_graphviz(dt_reg, out_file=None, 
                      feature_names=X.columns,   
                    filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render()

'Source.gv.pdf'

### This section explores random forest regressor to see its performance as compared to the Decision Tree

In [15]:
regr = RandomForestRegressor(max_depth=10, random_state=42)
regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))
y_pred = regr.predict(X_test)
print(mean_squared_error(y_test, y_pred))

0.6297081870699021
7653.924134818223


In [16]:
importance =pd.DataFrame(zip(X.columns, list(dt_reg.feature_importances_)), columns =['features', 'importance']).sort_values(by=['importance'],ascending =False)
importance['importance'] = importance['importance'].apply(lambda x: round(x,2))
importance

,features,importance
3,stringency_index,0.32
14,human_development_index,0.19
0,new_vaccinations_smoothed_per_million,0.12
8,gdp_per_capita,0.11
13,hospital_beds_per_thousand,0.07
4,population_density,0.06
11,cardiovasc_death_rate,0.04
7,aged_70_older,0.02
5,median_age,0.02
12,diabetes_prevalence,0.02


### This section explores the use of linear regression to model new COVID cases
Using the same set of variables, run a linear regression and use the results as a baseline for comparison against other models

In [26]:
reg = LinearRegression()
data = dataframe.fillna(0)
X = data[['new_vaccinations_smoothed_per_million'
          ,'people_vaccinated_per_hundred'
          ,'people_fully_vaccinated_per_hundred'
            ,'stringency_index'
            ,'population_density'
            ,'median_age'
            ,'aged_65_older'
            ,'aged_70_older'
            ,'gdp_per_capita'
            ,'extreme_poverty'
#             ,'female_smokers'
#             ,'male_smokers'
              ,'handwashing_facilities'
             ,'cardiovasc_death_rate'
             ,'diabetes_prevalence'
#             ,'hospital_beds_per_thousand'
#             ,'life_expectancy'
             ,'human_development_index'
#          ,'Covaxin', 'EpiVacCorona', 'Johnson&Johnson', 'Moderna',
#        'Oxford/AstraZeneca', 'Pfizer/ BioNTech', 'Pfizer/BioNTech',
#        'Sinopharm/Beijing', 'Sinopharm/Wuhan', 'Sinovac', 'Sputnik V'
         ]]
y = data['new_cases_smoothed_per_million']

    
linreg = reg.fit(X_train, y_train)
print(reg.score(X_test, y_test))


0.13116547155159675


In [19]:
importance =pd.DataFrame(zip(X.columns, list(dt_reg.feature_importances_)), columns =['features', 'importance']).sort_values(by=['importance'],ascending =False)
importance['importance'] = importance['importance'].apply(lambda x: round(x,2))
importance

,features,importance
3,stringency_index,0.21
0,new_vaccinations_smoothed_per_million,0.13
8,gdp_per_capita,0.13
4,population_density,0.07
13,human_development_index,0.05
11,cardiovasc_death_rate,0.05
7,aged_70_older,0.02
5,median_age,0.02
12,diabetes_prevalence,0.02
6,aged_65_older,0.01


### Analysis for new cases
The decision tree model is the best in predicting new COVID cases as compared to the random forest regression and the linear regression.

According to the decision tree model regressed against the New confirmed cases of COVID-19 (7-day smoothed), the most important factors affecting the number of new cases are -  **stringency index** - a composite measure based on 9 response indicators including school closures, workplace closures, and travel bans, rescaled to a value from 0 to 100 (100 = strictest response), **new vaccinations smoothed per million** and **gdp per capita**. This means that at this point, NPIs still play a huge role in affecting COVID cases, although new vaccinations help to tackle the spread of COVID, perhaps the proportion of the population  vaccinated is not enough to reach herd immunity. The proposed number by academia is 65-70% vaccinated to reach herd immunity. This might also be the reason why people vaccinated per hundred has a low feature importance score.

The brand of the vaccines appears to have low feature importance <= 0.01 in predicting the number of new cases, hence, it could be removed from the model to reduce complexity.

### This section explores the use of Decision Tree to model new COVID deaths per million
Using the same set of variables to predict new COVID deaths to compare the variable importance with the previous model which predicts new cases. To explore whether vaccinations rate and coverage affect deaths and cases differently.

In [25]:
dt_tree = DecisionTreeRegressor(max_depth =25,min_samples_split =20)
# reg = LinearRegression()
data = dataframe.fillna(0)
X = data[['new_vaccinations_smoothed_per_million'
          ,'people_vaccinated_per_hundred'
          ,'people_fully_vaccinated_per_hundred'
            ,'stringency_index'
            ,'population_density'
            ,'median_age'
            ,'aged_65_older'
            ,'aged_70_older'
            ,'gdp_per_capita'
            ,'extreme_poverty'
#             ,'female_smokers'
#             ,'male_smokers'
              ,'handwashing_facilities'
             ,'cardiovasc_death_rate'
             ,'diabetes_prevalence'
            ,'hospital_beds_per_thousand'
#             ,'life_expectancy'
             ,'human_development_index'
#          ,'Covaxin', 'EpiVacCorona', 'Johnson&Johnson', 'Moderna',
#        'Oxford/AstraZeneca', 'Pfizer/BioNTech',
#        'Sinopharm/Beijing', 'Sinopharm/Wuhan', 'Sinovac', 'SputnikV'
         ]]
y = data['new_deaths_smoothed_per_million']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
dt_reg = dt_tree.fit(X_train, y_train)
print(dt_reg.score(X_train, y_train))
dt_reg.score(X_test, y_test)

0.7814390663367289


0.7836452416008441

In [21]:
# view the importance of the variables for the new COVID deaths model
importance =pd.DataFrame(zip(X.columns, list(dt_reg.feature_importances_)), columns =['features', 'importance']).sort_values(by=['importance'],ascending =False)
importance['importance'] = importance['importance'].apply(lambda x: round(x,2))
importance

,features,importance
3,stringency_index,0.31
7,aged_70_older,0.13
0,new_vaccinations_smoothed_per_million,0.12
11,cardiovasc_death_rate,0.11
4,population_density,0.08
12,diabetes_prevalence,0.06
8,gdp_per_capita,0.06
14,human_development_index,0.03
6,aged_65_older,0.02
9,extreme_poverty,0.02


### Analysis for new deaths
As compared to the new cases model, the number of aged_70_older -for the proportion of population **aged 70 and older**  replaced **new vaccinations smoothed per million** indicating that COVID has a greater effect on the elderly, similarly cardiovascular death rate ranked 4th in the importance score implying that people who have health complications are more likely to die from COVID as shown by the prevailing studies. Vaccinations still play and importance role being 3rd in place. NPIs/ **stringency index** is ultimately the most important in both models. 